In [ ]:
parent_directory='/Users/jessicasun/Downloads/'

This notebook is used to visualize Fig S4, commensurate packings on a cylinder

In [ ]:
## general pipeline: 
## 1) run simulations to generate .dat files (~3 days, cluster)
## 2) process first half of this notebook to check for basic bugs (~1 day, local)
## 3) run analysis cell in the middle of this notebook to generate data_collect.pkl (~2 days, cluster)
## 4) finish analysis (defect_density_collect.pkl, etc) in this notebook (~1 day, local)
## 5) reimport relevant .pkl files and proceed with analysis (???? days)

##on cluster:
##import pickle5-> import pickle
##make sure run_dir_list_npy is correct
##run_dir_list=run_dir_list_npy
##input_dir and output_dir-> change to current (/n/manoharan/jessicasun instead of /Volumes/manoharan_lab/jessicasun)
# run_dir_df_filtered=run_dir_df[((run_dir_df['trial']>=((jobnum-1)*20))&(run_dir_df['trial']<(jobnum*20)))]


def start_timer():
    start=time.time()
    return start
def stop_timer(start,label,thresh=0.001):
    stop=time.time()
    time_elapsed=stop-start
    if time_elapsed>thresh:
        print(label,time_elapsed)
    return stop

#start=start_timer()
#start=stop_timer(start,'label') #the stop of this timer is start of next timer
#start=stop_timer(start,'label')

In [ ]:
from natsort import natsorted

In [ ]:
import matplotlib as mpl
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

import sympy
import seaborn as sns
import time
from scipy.stats import circmean
import pickle
###from bennett_plotter.ipynb
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D 
import mpl_toolkits.mplot3d.art3d as art3d
from math import atan2, cos, sin

input_dir=parent_directory+'dataverse_files/022323 bennett simulation cthetadeg0 magic mn' #load data from groupshare
# input_dir='/Volumes/manoharan_lab/jessicasun/Simulations/011422 bennett simulation' #load data from groupshare
output_dir=parent_directory+'cone-disk-packings-main/analysis_S4'
# output_dir='/Users/jessicasun/Documents/GitHub/cone-assembly-simulation/Code/jessica_bennett'
if not os.path.exists(output_dir+'/analysis'):
    os.mkdir(output_dir+'/analysis')
output_dir=output_dir+'/analysis'

# input_dir='/Users/jessicasun/Documents/GitHub/cone-assembly-simulation/Code/jessica_bennett'
os.chdir(input_dir)
# run_dir_list_npy=np.unique(np.load('run_dir_list_022322_0-99.npy'))
a=np.unique(np.load(parent_directory+'cone-disk-packings-main/simulation/simulation_inputs/'+'run_dir_list_022323_0-2cinit0-12.npy'))
b= np.unique(np.load(parent_directory+'cone-disk-packings-main/simulation/simulation_inputs/'+'run_dir_list_022323_0-2cinit0-12.npy'))
run_dir_list_npy=np.unique(np.concatenate((a,b)))
run_dir_list=[]

# ####################
# ##DIRECT IMPORT######
# ####################

# #if all files have been processed, you can directly set run_dir_list=run_dir_list_npy and comment out the below section

run_dir_list=run_dir_list_npy
# # with open(output_dir+'/run_dir_list.pkl', "rb") as fh: #read binary = rb, #file handle = fh
# #     run_dir_list = pickle.load(fh)
# #

# # #####################
# # ###QUICK IMPORT######
# # #####################
# # ##direct set without checking if run_dir contains dat file... much, much faster

# # # run_dir_list=os.listdir('datfiles/') 

####################
##THOROUGH IMPORT###
####################
# start=time.time()
# counter=0
# for run_dir in os.listdir('datfiles/'): #get directory names in datfiles
# #     if (os.path.exists('datfiles/'+run_dir+'/particles.dat'))&(os.path.exists('datfiles/'+run_dir+'/adj.dat')):    
#     if os.path.exists('datfiles/'+run_dir+'/particles.dat'):
#         run_dir_list+=[run_dir]
#         counter+=1
#         if counter%100==1:
#             print('.',end='')
# print('\n'+str(round(len(run_dir_list)/len(run_dir_list_npy)*100,2))+'%',end=' | ')
# print(str(len(run_dir_list))+' processed, '+str(len(run_dir_list_npy)-len(run_dir_list))+' remaining',end=' | ')
# print('load time: '+str(round((time.time()-start)/60,2))+'min\n')

# #### save as pickle and reimport so that we can work with this dataset snapshot####
# with open(output_dir+'/run_dir_list.pkl', 'wb') as f:
#     pickle.dump(run_dir_list, f)
# with open(output_dir+'/run_dir_list.pkl', "rb") as fh: #read binary = rb, #file handle = fh
#     run_dir_list = pickle.load(fh)

In [ ]:
with open(parent_directory+'cone-disk-packings-main/analysis_S4'+'/cylinder_params_df.pkl', "rb") as fh:
    cylinder_params_df=pickle.load(fh)
cylinder_params_df

In [ ]:
def get_params_dict(run_dir):
    keys=[]
    values=[]
    for params in run_dir.split('_'):
        key,value=params.split('-')
        keys+=[key]
        values+=[float(value)]
    params_dict=dict(zip(keys,values))
    if params_dict['cthetadeg']==0:
#         params_dict['cthetadeg']=(params_dict['cthetadeg']*np.pi/180+0.002)*180/np.pi #otherwise it'll break down (division by 0 later, etc)
        params_dict['run_dir']=run_dir
        params_dict['yinit']=params_dict['cinit']/(0.002) #from simulation definition
        params_dict['R']=params_dict['yinit']+20 #from simulation definition
        params_dict['thetadeg']=0
    else:
#         params_dict['cthetadeg']=params_dict['cthetadeg']
        params_dict['run_dir']=run_dir
        params_dict['yinit']=params_dict['cinit']/(params_dict['cthetadeg']*np.pi/180) #from simulation definition
        params_dict['R']=params_dict['yinit']+20 #from simulation definition
        params_dict['thetadeg']=2*ctheta_to_alpha(params_dict['cthetadeg']*np.pi/180)*180/np.pi
    return params_dict
def get_run_dir_df(run_dir_list):
#     run_dir_df=pd.DataFrame() #DO NOT GROW A DATAFRAME!!!! grow a list of small df instead
    run_dir_df_list=[]
    for run_dir in run_dir_list:
        params_dict=get_params_dict(run_dir)
        run_dir_df_list+=[pd.DataFrame([params_dict])]
#         run_dir_df=run_dir_df.append(pd.DataFrame([params_dict]))
    run_dir_df=pd.concat(run_dir_df_list,axis=0)
    return run_dir_df
def run_dir_df_info(run_dir_df):
    print('input_dir','\n',input_dir)
    print('\noutput_dir','\n',output_dir,'\n')
    info_list=[]
    for item in run_dir_df.columns[-1:]:
        print(item,'\n',np.unique(run_dir_df[item]),'\n')
        info_list+=[str(item)+'-'+str(int(np.min(np.unique(run_dir_df[item]))))+'-'+str(int(np.max(np.unique(run_dir_df[item]))))]
    for item in run_dir_df.columns[:5]:
        print(item,'\n',np.unique(run_dir_df[item]),'\n')
        info_list+=[str(item)+'-'+str(int(np.min(np.unique(run_dir_df[item]))))+'-'+str(int(np.max(np.unique(run_dir_df[item]))))]
    info_tag='_'.join(info_list)
    print('info_tag','\n',info_tag,'\n')
    return info_tag
def ctheta_to_alpha(cthetarad):
    # alpha=theta/2=arcsin(phi/(2pi)), theta=full cone angle, alpha=half cone angle, phi=sector angle
    alpharad=np.arcsin(cthetarad/(2*np.pi))
    return alpharad
def load_dat(run_dir):
#     particles=np.loadtxt('datfiles/'+run_dir+'/particles.dat')
    particles=pd.read_csv('datfiles/'+run_dir+'/particles.dat', sep=' ', header=None).to_numpy()
#     adj=np.loadtxt('datfiles/'+run_dir+'/adj.dat')
    adj=pd.read_csv('datfiles/'+run_dir+'/adj.dat', sep=' ', header=None).to_numpy()
    return particles,adj
def find_coordnum(particles,adj):
    energy_list=[]
    for particle_idx in range(len(particles)):
        energy=sum(adj[:,particle_idx])
        energy_list+=[energy]
    data=pd.concat([pd.DataFrame(particles,columns=['x','y']),pd.DataFrame(energy_list,columns=['coordnum'])],axis=1)    
    return data,adj
def load_data(run_dir,shifted=True): #shifted moves the seed to the side so that seam is likely at center
    particles,adj=load_dat(run_dir)
    data,adj=find_coordnum(particles,adj)
    if shifted:
        data=get_data_shifted(data,4/3,params_dict['cthetadeg'])        
    return data,adj
def load_data_slice(run_dir,n,shifted=True): #number of particles to load (indices are in order of placement)
    #this is ok if just loading a few slices, but if you're going to load ALL slices, then it's better to not use this function
    #because this function forces load_dat for EACH slice which is unnecessary.
    #instead,load_dat first, then loop through particles=particles[:n],and then find_coordnum.
    particles,adj=load_dat(run_dir)    
    particles,adj=particles[:n],adj[:n]
    data_slice,adj=find_coordnum(particles,adj)
    if shifted:
        data_slice=get_data_shifted(data_slice,4/3,params_dict['cthetadeg'])        
    return data_slice,adj

def rotate_point_shift(r,theta,rotatethetadeg):
    rotated_theta=theta+rotatethetadeg*np.pi/180
    y=r*np.sin(rotated_theta)
    x=r*np.cos(rotated_theta)
    return x,y,rotated_theta
def plot_data_shifted(data_shifted):
    fig=plt.figure(figsize=(15,15))
    ax=fig.add_subplot(131)
    ax.scatter(data_shifted['x'],data_shifted['y'])
    ax.scatter(data_shifted['x'][:3],data_shifted['y'][:3])
    ax.set_aspect('equal')
    ax.invert_yaxis()    
    ax.set_title('unshifted data')

    ax=fig.add_subplot(132)
    ax.scatter(data_shifted['x_rotated'],data_shifted['y_rotated'])
    ax.scatter(data_shifted['x_rotated'][:3],data_shifted['y_rotated'][:3])
    ax.set_aspect('equal')
    ax.invert_yaxis()    
    ax.set_title('rotated data')

    ax=fig.add_subplot(133)
    ax.scatter(data_shifted['x_final'],data_shifted['y_final'])
    ax.scatter(data_shifted['x_final'][:3],data_shifted['y_final'][:3])
    ax.set_aspect('equal')
    ax.invert_yaxis()
    ax.set_title('final shifted data')
def get_data_shifted(data, distfromedge,cthetadeg,plot_data_shifted_bool=False):
    if params_dict['cthetadeg']!=0:
        rotatethetadeg=(params_dict['cthetadeg']/2)-((distfromedge)/(params_dict['cinit']/(params_dict['cthetadeg']*np.pi/180)))*180/np.pi #(4/3)/(cinit/(cthetadeg*np.pi/180)) ->4/3 is arclength distance from edge, can be adjusted for display purposes
        data_shifted=pd.DataFrame([])
        data_shifted['x']=data['x']
        data_shifted['y']=data['y']
        data_shifted['r']=np.sqrt(data_shifted['x']**2+data_shifted['y']**2)
        data_shifted['theta']=np.arctan2(data_shifted['y'],data_shifted['x'])

        ctheta=cthetadeg*np.pi/180
        data_shifted['x_rotated'],data_shifted['y_rotated'],data_shifted['theta_rotated']=rotate_point_shift(data_shifted['r'],data_shifted['theta'],rotatethetadeg)

        data_shifted['theta_final']=np.zeros(len(data_shifted['theta_rotated'])) #initialize column
        data_shifted['x_final']=np.zeros(len(data_shifted['theta_rotated'])) #initialize column
        data_shifted['y_final']=np.zeros(len(data_shifted['theta_rotated'])) #initialize column
        #   apply pbc. include the RHS in the domain
        for idx in range(len(data_shifted)):
            angle=data_shifted['theta_rotated'][idx]
            r=data_shifted['r'][idx]
            #include the RHS in the domain
            if angle>(-np.pi/2.0+ctheta/2.0):
                angle=angle-np.floor((np.abs(angle-(-np.pi/2))+ctheta*0.5)/ctheta)*ctheta
            if angle<=(-np.pi/2.0-ctheta/2.0):
                angle=angle+np.floor((np.abs(angle-(-np.pi/2))+ctheta*0.5)/ctheta)*ctheta
            data_shifted['theta_final'][idx]=angle
            data_shifted['x_final'][idx]=r*cos(angle)
            data_shifted['y_final'][idx]=r*sin(angle)

        #plot to make sure everything looks ok
        if plot_data_shifted_bool:
            plot_data_shifted(data_shifted)

        #transfer extra info over, reformat to match original 'data'
        data_shifted_reformat=pd.DataFrame([])
        data_shifted_reformat['x']=data_shifted['x_final']
        data_shifted_reformat['y']=data_shifted['y_final']
        data_shifted_reformat['coordnum']=data['coordnum']
    if params_dict['cthetadeg']==0:
        Lx=params_dict['cinit']
        rotatethetadeg=(Lx/2)-(distfromedge)
        data_shifted=pd.DataFrame([])
        data_shifted['x']=data['x']
        data_shifted['y']=data['y']

        data_shifted['x_rotated']=data_shifted['x']+rotatethetadeg
        data_shifted['y_rotated']=data_shifted['y']

        data_shifted['x_final']=np.zeros(len(data_shifted['x'])) #initialize column
        data_shifted['y_final']=np.zeros(len(data_shifted['x'])) #initialize column
        #   apply pbc. include the RHS in the domain
        for idx in range(len(data_shifted)):            
            x,y=data_shifted['x_rotated'][idx],data_shifted['y_rotated'][idx]
            if x>Lx/2.0:
                x=x-np.floor((x+Lx*0.5)/(Lx))*Lx
            if x<=-Lx/2.0:
                x=x+np.floor(abs(x-Lx*0.5)/Lx)*Lx   

            data_shifted['x_final'][idx]=x
            data_shifted['y_final'][idx]=y

        #plot to make sure everything looks ok
        if plot_data_shifted_bool:
            plot_data_shifted(data_shifted)

        #transfer extra info over, reformat to match original 'data'
        data_shifted_reformat=pd.DataFrame([])
        data_shifted_reformat['x']=data_shifted['x_final']
        data_shifted_reformat['y']=data_shifted['y_final']
        data_shifted_reformat['coordnum']=data['coordnum']        
    return data_shifted_reformat

print('get run_dir_df...')
run_dir_df=get_run_dir_df(run_dir_list)
run_dir_df_info(run_dir_df)
run_dir_df.head()

print('done!')

In [ ]:
# # # run import modules and this cell at beginning of new session

# # ##########
# # ##### takes around 1 hr (6min if you're lucky)
# print('opening defect_density_collect.pkl...')
# start=time.time()
# with open(output_dir+'/defect_density_collect.pkl', "rb") as fh:
#     defect_density_collect=pickle.load(fh)
# print('done!',round((time.time()-start)/60,2),'min import')

# # print('opening defect_count_collect.pkl...')
# # start=time.time()
# # with open(output_dir+'/defect_count_collect.pkl', "rb") as fh:
# #     defect_count_collect=pickle.load(fh)
# # print('done!',round((time.time()-start)/60,2),'min import')

# # print('opening data_count_collect.pkl...')
# # start=time.time()
# # with open(output_dir+'/data_count_collect.pkl', "rb") as fh:
# #     data_count_collect=pickle.load(fh)
# # print('done!',round((time.time()-start)/60,2),'min import')
# # ##########
# # ##### takes around 6hr (30min if you're lucky)
# # print('opening data_collect_compiled.pkl...')
# # start=time.time()
# # with open(output_dir+'/data_collect_compiled.pkl', "rb") as fh:
# #     data_collect_compiled=pickle.load(fh)
# # print('done!',round((time.time()-start)/60,2),'min import')

Fig. 1: Geometric frustration schemes - 3D, 2D, 2D curved surface in 3D.

Fig. 2: Unwrapping into 2D. UV mapping procedure. Maybe there’s a more technical way of doing this.

Fig. 3: magic cone, varying cinit. cthetadeg=60, ithetadeg=30, trial=0. cinit=7, 7.225…, 7.330…

Fig. 4: magic cone, varying ithetadeg. cthetadeg=60, ithetadeg=0,10,20, trial=0. cinit=7.225…

Fig 5. different kinds of defect structures. stacking faults and pure disorder. cinit=7/itheta=20; cinit=6.35, itheta=60 (ideally for consistency, find one for cinit=7-7.330 range, even if it’s a diff trial)… color coded by psi6. 

Fig 6. magic cone. average psi6 vs ithetadeg 0-60. cinit=7, cinit=7.225, cinit=7.330.

Fig 7. Defect density vs cinit. plot of defect density vs cinit

Fig 8. Finite size effects appear for small circumferences. cylinder, itheta=30deg. Plot of defect density vs cinit for cylinders.

Fig 9. Large circ cone initialize vs small circ cone initialize

Fig 10. defect density vs r and defect density vs circ. only show range 5-30degrees, at enlarge cinit.

Fig 11. Large cinit vs small cinit: same plot just with dip

In [ ]:
#############
####FIG 1####
#############

#make separately

In [ ]:
def get_newcmp(origcmap='plasma',newcmapname='newcmp',cyclic=False,crop=False,bottom=0,top=245,replace=True,r=100,g=100,b=100,a=1):
    newcolors = cm.get_cmap(origcmap, 256)(np.linspace(0, 1, 256))
    if crop:
        newcolors=newcolors[bottom:top,:]
    if replace:
        if not cyclic:
            newcolors[-25:, :] = np.array([r/256, g/256, b/256, a]) #165, 140, 0, 0.8
        if cyclic:
            newcolors[-25:, :] = np.array([r/256, g/256, b/256, a]) #165, 140, 0, 0.8
            newcolors[:25, :] = np.array([r/256, g/256, b/256, a]) #165, 140, 0, 0.8        
    newcmp = ListedColormap(newcolors)
    plt.register_cmap(name=newcmapname,cmap=newcmp)    
    return newcmp
def standalone_cbar(cmap='RdBu',orientation='vertical',discrete=False,n_clusters=5,vmin=0,vmax=6,title='title'):
    fig = plt.figure(figsize=(2,2),dpi=400)
    if orientation=='vertical':
        ax = fig.add_axes([0.05, 0.80, 0.05, 0.9]) #vertical [left bottom width height]
    if orientation=='horizontal':
        ax = fig.add_axes([0.05, 0.80, 0.9, 0.05]) #horizontal [left bottom width height]   
    
    if not discrete:
        cmap=plt.get_cmap(cmap)
        
        cb = mpl.colorbar.ColorbarBase(ax, orientation=orientation, 
                                       cmap=cmap,norm=mpl.colors.Normalize(vmin=vmin, vmax=vmax))
    if discrete:
        n_clusters=(vmax-vmin)+1
        cmap=plt.get_cmap(cmap, n_clusters)
        
        
        cb = mpl.colorbar.ColorbarBase(ax, orientation=orientation, 
                                       cmap=cmap,norm=mpl.colors.Normalize(vmin=vmin, vmax=vmax))
                
        tick_locs = (np.arange(vmin,vmax+1,(vmax-vmin)/n_clusters))+0.5*((vmax-vmin)/n_clusters)
        cb.set_ticks(tick_locs[:-2])
        cb.set_ticklabels(np.arange(vmin,vmax+1,1))
        
    cb.outline.set_visible(False)
    cb.ax.tick_params(size=0)
    cb.ax.set_title(title)
#     plt.savefig(output_dir+'/cbar'+'.png',bbox_inches='tight')  

# if c=='coordnum':
#     vmin,vmax=2,6
# if c=='psi6':
#     vmin,vmax=0,1
# if c=='theta6':
#     vmin,vmax=0,60

newcmp=get_newcmp(origcmap='plasma',newcmapname='newcmp',cyclic=False,r=180,g=180,b=180,a=0.4) #for sector plots
# get_newcmp(origcmap='plasma',newcmapname='newplasma',cyclic=False,crop=True,bottom=0,top=225,replace=False,r=100,g=100,b=100,a=1) #for line plots
get_newcmp(origcmap='viridis',newcmapname='newviridis',cyclic=False,crop=True,bottom=0,top=225,replace=False,r=100,g=100,b=100,a=1) #for line plots
# get_newcmp(origcmap='twilight',newcmapname='newtwilight',cyclic=True,crop=True,bottom=30,top=225,replace=False,r=100,g=100,b=100,a=1) #for line plots
get_newcmp(origcmap='cividis',newcmapname='newcividis',cyclic=False,crop=True,bottom=0,top=225,replace=False,r=100,g=100,b=100,a=1) #for line plots

# standalone_cbar(cmap='newcmp',orientation='horizontal',discrete=False,vmin=2,vmax=6,title='newcmp')
# standalone_cbar(cmap='newplasma',orientation='horizontal',discrete=False,vmin=2,vmax=6,title='newplasma')
# standalone_cbar(cmap='newviridis',orientation='horizontal',discrete=False,vmin=2,vmax=6,title='newviridis')
# standalone_cbar(cmap='newtwilight',orientation='horizontal',discrete=False,vmin=2,vmax=6,title='newtwilight')
# standalone_cbar(cmap='newcividis',orientation='horizontal',discrete=False,vmin=2,vmax=6,title='newcividis')

standalone_cbar(cmap='newcmp',orientation='horizontal',discrete=True,vmin=2,vmax=6,title=r'$N_{j}$')
# plt.savefig('/Users/jessicasun/Documents/GitHub/062122-bennett-draft/main_figs/figure_components_svg/newcmp_horizontal.svg',bbox_inches = "tight")
standalone_cbar(cmap='newcmp',orientation='vertical',discrete=True,vmin=2,vmax=6,title=r'$N_{j}$')
# plt.savefig('/Users/jessicasun/Documents/GitHub/062122-bennett-draft/main_figs/figure_components_svg/newcmp_vertical.svg',bbox_inches = "tight")
# standalone_cbar(cmap='viridis',orientation='vertical',discrete=True,vmin=2,vmax=6,title=r'$N_{ij}$')
# standalone_cbar(cmap='plasma',orientation='horizontal',discrete=False,vmin=0,vmax=1,title=r'$\psi_6$')
# standalone_cbar(cmap='hsv',orientation='horizontal',discrete=False,vmin=0,vmax=60,title=r'$\theta$')

In [ ]:
cmap=newcmp
#############
####FIG 2####
#############
def uv_wrap(x2D,y2D,ctheta_rad):
    if ctheta_rad!=0:
        x3D=[]
        y3D=[]
        z3D=[]
        for x,y in zip(x2D,y2D): #x,y describe coordinates of flat net of particles
            u,v=[(atan2(y,x)-np.pi/2+ctheta_rad/2)*(2*np.pi/ctheta_rad),np.sqrt(x**2+y**2)] #theta from (0 to +ctheta) then normalized and mapped onto 2pi, r distance from origin
            arclength=v*ctheta_rad #s=r*theta. arclength of sector is circumference of wrapped cone at given point
            cone_r=(arclength/np.pi)/2.0 #radius of 3D CONE, not the radial coord of flat net sector.
            cone_alpha=np.arcsin(ctheta_rad/(2*np.pi)) #alpha is half cone angle (angle of slant from cone vertical). use cone equation from first slide of prof nelson's pdf.
            cone_z=cone_r/np.tan(cone_alpha) #find height of cone. cone_r is radius, v is slant length.             
            x3D+=[cone_r*np.cos(u)] #convert u,v to x,y,z.
            y3D+=[cone_r*np.sin(u)] #convert u,v to x,y,z.
            z3D+=[cone_z] #convert u,v to x,y,z.
    if ctheta_rad==0:
        Lx=params_dict['cinit']
        x3D=[]
        y3D=[]
        z3D=[]
        for x,y in zip(x2D,y2D): #x,y describe coordinates of flat net of particles
            u=(x/Lx)*2*np.pi-np.pi/2
            v=-y            
            #######
            arclength=Lx #s=r*theta. arclength of sector is circumference of wrapped cone at given point
            cone_r=(arclength/np.pi)/2.0 #radius of 3D CONE, not the radial coord of flat net sector.
#             cone_alpha=np.arcsin(ctheta_rad/(2*np.pi)) #alpha is half cone angle (angle of slant from cone vertical). use cone equation from first slide of prof nelson's pdf.
            cone_z=v #find height of cone. cone_r is radius, v is slant length.             
            x3D+=[cone_r*np.cos(u)] #convert u,v to x,y,z.
            y3D+=[cone_r*np.sin(u)] #convert u,v to x,y,z.
            z3D+=[cone_z] #convert u,v to x,y,z.        
    return x3D,y3D,z3D
def plot_sector(data,params_dict,ax,alpha=1,c='coordnum',cmap='viridis'):
    if c=='coordnum':
        vmin,vmax=2,6
    if c=='psi6':
        vmin,vmax=0,1
    if c=='theta6':
        vmin,vmax=0,60
    c=data[c]
    
    # fig=plt.figure(figsize=(5,5),dpi=400)
    if params_dict['cthetadeg']!=0:
        ax.add_patch(patches.Wedge((0,0), -params_dict['R'], 90-params_dict['cthetadeg']/2, 90+params_dict['cthetadeg']/2,fill=False,edgecolor='black',linewidth=0.25))
    if params_dict['cthetadeg']==0:
        Lx=params_dict['cinit']
        Ly=params_dict['R']
        ax.add_patch(patches.Rectangle((-Lx/2,0),Lx,-params_dict['R'],fill=False,edgecolor='black',linewidth=0.25))        
    ax.scatter(data['x'],data['y'],c=c,s=1,alpha=alpha,cmap=cmap,vmin=vmin,vmax=vmax) #plot points
    try:
        ax.scatter(data.loc[0]['x'],data.loc[0]['y'],color='black',s=1,alpha=alpha,zorder=200) #plot seed
        ax.scatter(data.loc[1]['x'],data.loc[1]['y'],color='black',s=1,alpha=alpha,zorder=200) #plot seed
        ax.scatter(data.loc[2]['x'],data.loc[2]['y'],color='black',s=1,alpha=alpha,zorder=200) #plot seed    
    except:
        pass
    ax.set_xlim((np.min(data['x'])-10,np.max(data['x'])+10))
    ax.set_ylim((np.min(data['y'])-10,np.max(data['y'])+10))
    ax.set_aspect('equal')
    # ax.set_title(run_dir+'\n'+'thetadeg-'+str(round(params_dict['thetadeg'],1))+'_yinit-'+str(round(params_dict['yinit'],1))+'_R-'+str(round(params_dict['R'],1)),fontsize=5)
    ax.axis('off')
    ax.invert_yaxis()
def plot_cone(data,params_dict,ax,planealpha=0.85,elev_val=0,azim_val=180,alpha=1,cmap='viridis'):
    #plot semitransparent plane. we do this instead of setting alpha of point markers because matplotlib does a weird gradient for transparent markers
    zz,yy = np.meshgrid(range(int(np.min(data['z3D'])-10),int(np.max(data['z3D'])+10)), range(int(np.min(data['y3D'])-10),int(np.max(data['y3D'])+10)))
    xx = yy*0
    ax.plot_surface(xx, yy, zz,alpha=planealpha,color='w',shade=False,antialiased=False,linewidth=0,edgecolor=None,rstride=100,cstride=100) #if the alpha set for overall figure is 1, have a semitransparent plane. otherwise set the plane alpha value to half of that of the datapoint alpha value (input).
        
    #set axis limits based on data bounds
    ax.set_xlim((np.min(data['x3D'])-2,np.max(data['x3D'])+2))
    ax.set_ylim((np.min(data['y3D'])-2,np.max(data['y3D'])+2))
    ax.set_zlim((np.min(data['z3D'])-2,np.max(data['z3D'])+2))    

    #     ax.set_aspect('equal')
    ax.set_box_aspect((ax.get_xlim()[1]-ax.get_xlim()[0],ax.get_ylim()[1]-ax.get_ylim()[0],ax.get_zlim()[1]-ax.get_zlim()[0]))

    
    #filter points based on front or back
    data_back=data[data['x3D']>=0] #back of 3D cone
    data_front=data[data['x3D']<0] #front of 3D cone
    data=data_back #i use data.loc which is label-based later, so it won't plot seeds if it's not in the range
    # fig=plt.figure(figsize=(4,4),dpi=400)
    #     ax.add_patch(patches.Wedge((0,0), -params_dict['R'], 90-params_dict['cthetadeg']/2, 90+params_dict['cthetadeg']/2,fill=False,edgecolor='black',linewidth=0.25))
    data_list=[(data_back,alpha*1), (data_front,alpha)] #alpha controls POINTS alpha
    
    #plot points based on front or back
    for data,alpha in data_list:
        try:
            #plot datapoints
            ax.scatter3D(data.loc[3:]['x3D'],data.loc[3:]['y3D'],data.loc[3:]['z3D'],c=data.loc[3:]['coordnum'],s=1,alpha=alpha,cmap=cmap,vmin=2,vmax=6) #plot points
            ax.scatter3D(data.loc[0:2]['x3D'],data.loc[0:2]['y3D'],data.loc[0:2]['z3D'],color='black',s=1,alpha=alpha,vmin=2,vmax=6) #plot seed               
        except:
            continue
    # ax.set_title(run_dir+'\n'+'thetadeg-'+str(round(params_dict['thetadeg'],1))+'_yinit-'+str(round(params_dict['yinit'],1))+'_R-'+str(round(params_dict['R'],1)),fontsize=5)
    ax.axis('off')
    ax.invert_yaxis()
    ax.invert_xaxis()
    
#     ax.set_xlabel('x')
#     ax.set_ylabel('y')    
#     ax.set_zlabel('z')    
    ax.view_init(elev=elev_val,azim=azim_val)      
def plot_seed(data,params_dict,alpha=1,c='coordnum',cmap='viridis'):
    fig=plt.figure(figsize=(1.5,1.5),dpi=400)
    ax=fig.add_subplot(111)    
    #seed vectors
    data=data[:3] #take only seeds of the dataset
    seed0x, seed0y=data.loc[0]['x'],data.loc[0]['y']
    seed1x, seed1y=data.loc[1]['x'],data.loc[1]['y']
    seed2x, seed2y=data.loc[2]['x'],data.loc[2]['y']
    
    if c=='coordnum':
        vmin,vmax=2,6
    if c=='psi6':
        vmin,vmax=0,1
    if c=='theta6':
        vmin,vmax=0,60
    c=data[c]
    seedR=np.sqrt(seed0x**2+seed0y**2)
    # fig=plt.figure(figsize=(5,5),dpi=400)
#     if params_dict['cthetadeg']!=0:
# #         ax.add_patch(patches.Wedge((0,0), -seedR, 90-params_dict['cthetadeg']/2, 90+params_dict['cthetadeg']/2,fill=False,edgecolor='black',linewidth=0.25))
#         ax.add_patch(patches.Arc((0,0), -2*seedR,-2*seedR, angle=0,theta1=90-params_dict['cthetadeg']/2-10, theta2=90+params_dict['cthetadeg']/2+10,fill=False,edgecolor='black',linestyle='--',linewidth=0.25))
#     if params_dict['cthetadeg']==0:
#         Lx=params_dict['cinit']
#         Ly=params_dict['R']
#         ax.add_patch(patches.Rectangle((-Lx/2,0),Lx,-params_dict['R'],fill=False,edgecolor='black',linewidth=0.25))
#     ax.scatter(data['x'],data['y'],c=c,s=1,alpha=alpha,cmap=cmap,vmin=vmin,vmax=vmax) #plot points
    try:
        ax.scatter(seed0x,seed0y,color='black',s=50,alpha=alpha) #plot seed
        ax.scatter(seed1x,seed1y,color='black',s=50,alpha=alpha) #plot seed
        ax.scatter(seed2x,seed2y,color='black',s=50,alpha=alpha) #plot seed    

#         ax.annotate("", xy=(2*(seed1x-seed0x)+seed0x,seed0y), xytext=(-2*(seed1x-seed0x)+seed0x, seed0y),arrowprops=dict(arrowstyle="-",color='0.3'),zorder=0) #plot ref line        
        ellipsewidth=5*(seed1x-seed0x)
        ellipseheight=0.5
        ellipse=mpl.patches.Ellipse((seed0x,seed0y-ellipseheight/2),ellipsewidth,ellipseheight, angle=0,zorder=0,edgecolor='0.3',linewidth=1,facecolor='none')        
        ax.add_patch(ellipse)
        ax.annotate("", xy=(2*(seed1x-seed0x)+seed0x,2*(seed1y-seed0y)+seed0y), xytext=(seed0x, seed0y),arrowprops=dict(arrowstyle="->",linewidth=0.5)) #plot arrow
        ax.annotate("", xy=(-2*(seed1x-seed0x)+seed0x,-2*(seed1y-seed0y)+seed0y), xytext=(seed0x, seed0y),arrowprops=dict(arrowstyle="->",linewidth=0.5)) #plot arrow        
        
        ax.annotate("", xy=(2*(seed2x-seed0x)+seed0x,2*(seed2y-seed0y)+seed0y), xytext=(seed0x, seed0y),arrowprops=dict(arrowstyle="->",linewidth=0.5)) #plot arrow
        ax.annotate("", xy=(-2*(seed2x-seed0x)+seed0x,-2*(seed2y-seed0y)+seed0y), xytext=(seed0x, seed0y),arrowprops=dict(arrowstyle="->",linewidth=0.5)) #plot arrow        

        ax.annotate("", xy=(2*(seed2x-seed1x)+seed0x,2*(seed2y-seed1y)+seed0y), xytext=(seed0x, seed0y),arrowprops=dict(arrowstyle="->",linewidth=0.5)) #plot arrow
        ax.annotate("", xy=(-2*(seed2x-seed1x)+seed0x,-2*(seed2y-seed1y)+seed0y), xytext=(seed0x, seed0y),arrowprops=dict(arrowstyle="->",linewidth=0.5)) #plot arrow                
#     #rotate vector to find the reference line
#     initx=seed1x - seed0x
#     inity=seed1y - seed0y
#     refx, refy = initx*np.cos(params_dict['ithetadeg']) - inity*np.sin(params_dict['ithetadeg']), initx*np.sin(params_dict['ithetadeg']) + inity*np.cos(params_dict['ithetadeg'])    
#     refx, refy = refx+seed0x, refy+seed0y
        
#         ax.annotate("", xy=(2*(refx-seed0x)+seed0x,2*(refy-seed0y)+seed0y), xytext=(seed0x, seed0y),arrowprops=dict(arrowstyle="->",color='grey')) #plot arrow        
    except:
        pass
    ax.set_xlim(seed0x-3,seed0x+3)
    ax.set_ylim((seed0y-3),seed0y+3)
    ax.set_aspect('equal')
    # ax.set_title(run_dir+'\n'+'thetadeg-'+str(round(params_dict['thetadeg'],1))+'_yinit-'+str(round(params_dict['yinit'],1))+'_R-'+str(round(params_dict['R'],1)),fontsize=5)
    ax.axis('off')
    ax.invert_yaxis()

for run_dir in run_dir_df[run_dir_df['trial']==0]['run_dir'].tolist()[::-1]:
    run_dir_params=run_dir_df[run_dir_df['run_dir']==run_dir]
    cthetadeg=run_dir_params['cthetadeg'].values[0]
    cinit=run_dir_params['cinit'].values[0]
    ithetadeg=run_dir_params['ithetadeg'].values[0]
    trial=run_dir_params['trial'].values[0]

#     run_dir_df_filtered=run_dir_df[
#         (run_dir_df['cthetadeg']==cthetadeg)
#         &(run_dir_df['cinit']==cinit)
#         &(run_dir_df['ithetadeg']==ithetadeg)
#         &(run_dir_df['trial']==trial)]

#     run_dir=run_dir_df_filtered['run_dir'].iloc[0]
    params_dict=get_params_dict(run_dir)
    data,adj=load_data(run_dir)
    data['x3D'],data['y3D'],data['z3D']=uv_wrap(data['x'],data['y'],params_dict['cthetadeg']*np.pi/180)

    
    reference_range=40 #temporary set
    fig=plt.figure(figsize=(6.5,3),dpi=400)
    ax=fig.add_subplot(121)
    plot_sector(data,params_dict,ax,cmap=cmap)
    Nactual=len(data)
    info_tag='_cthetadeg-'+str(cthetadeg)+'_cinit-'+str(cinit)+'_ithetadeg-'+str(ithetadeg)+'_trial-'+str(trial)+'_Nactual-'+str(Nactual)
    ax.set_ylim(min(data['y'])-10,min(data['y'])+reference_range+50)
    ax.invert_yaxis()
    # plt.savefig(output_dir+'/fig2a'+info_tag+'.png')
    # plt.savefig('/Users/jessicasun/Documents/GitHub/062122-bennett-draft/main_figs/figure_components_svg/fig1sector.svg',bbox_inches = "tight")

    # fig=plt.figure(figsize=(3,3),dpi=400)
    ax1=fig.add_subplot(122,projection='3d')
    plot_cone(data,params_dict,ax1,azim_val=180,cmap=cmap)
    # ax.set_zlim(min(data['z3D'])-10,min(data['z3D'])+reference_range+50)    
    ax1.set_zlim(max(data['z3D'])-reference_range-50,max(data['z3D'])+10)        
    # plot_circ(data,params_dict,ax,alpha=1,c='coordnum',cmap='viridis')
    # plot_cone(data,params_dict,ax,alpha=0,planealpha=0,azim_val=180,elev_val=10) #set bounds

    #each cylinder plot can be indexed by (C, theta) and trial. theta stores (m,n) info for parastichy number.
    #check if the cinit of our data is at a orientation that should result in commensurate packing
    suptitle='C='+str(round(cinit,3))+', '+r'$\theta_{seed}$='+str(round(ithetadeg,3))+r'$^\circ$'+'\n'
    magic_info=cylinder_params_df[(cylinder_params_df['cinit']==cinit)]
    if ithetadeg==magic_info['ithetadeg'].values[0]:
        fig.patch.set_facecolor("lightyellow") #ax has no axis anymore so set the fig background instead...
        ax1.set_facecolor("lightyellow") #highlight if its ~magic~        
        c=str(round(cinit,3))
        theta_seed=str(round(ithetadeg,3))+r'$^\circ$'
        mn='('+str(magic_info['m'].values[0])+','+str(magic_info['n'].values[0])+')'
        a=str(magic_info['a'].values[0])
        phi_seed=str(round(magic_info['phi_dbeller_deg'].values[0],3))+r'$^\circ$'
        suptitle='C='+c+', '+r'$\theta_{seed}$='+theta_seed+'\n'+'(m,n)='+mn+', a='+a+', '+r'$\phi_{seed}$='+phi_seed
        
    plt.suptitle(suptitle)
    # # info['']
    # cylinder_params_df[]
    fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)

    if not os.path.exists(output_dir+'/png'):
        os.mkdir(output_dir+'/png')
    
    plt.savefig(output_dir+'/png/C-'+str(round(cinit,3))+'_theta-'+str(round(ithetadeg,3))+'.png',bbox_inches = "tight")
    plt.show()

In [ ]:
png_df_list=[]
for png_dir in os.listdir(output_dir+'/png/'):
    if png_dir.endswith('.png'):
        C=png_dir.split('.png')[0].split('_')[0].split('-')[1]
        theta=png_dir.split('.png')[0].split('_')[1].split('-')[1]
        png_df=pd.DataFrame([{'png_dir':png_dir,
                            'C':C,
                            'theta':theta}])
        png_df_list+=[png_df]
png_df=pd.concat(png_df_list)

In [ ]:
png_df

In [ ]:
y=natsorted(np.unique(png_df['C']))[::-1]
x=natsorted(np.unique(png_df['theta']))[::-1]

fig, axs = plt.subplots(ncols=len(x), nrows=(len(y)),figsize=(25,75))

for i in range(len(x)):
    for j in range(len(y)):
        ax=axs[j,i]
        theta=x[i]
        C=y[j]
        png_dir=png_df[(png_df['C']==C)&(png_df['theta']==theta)]['png_dir'].values[0]
        img=plt.imread(output_dir+'/png/'+png_dir)
        ax.imshow(img)
        ax.axis('off')
plt.tight_layout()
plt.savefig('analysis/overview.png', dpi=600,bbox_inches='tight', pad_inches=0)
plt.show()